In [3]:
import pandas as pd
import sqlite3

In [4]:
%run ./transform_and_create_tables.ipynb

<class 'str'>


/tmp/ipykernel_74156/3054090517.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_originacao.VLCONTRATO = base_originacao.VLCONTRATO.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
/tmp/ipykernel_74156/3054090517.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_originacao.VLPARCELA = base_originacao.VLPARCELA.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
/tmp/ipykernel_74156/3054090517.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_carteira.VLCONTRATO =

contém 0 registros duplicados na base_carteira!
contém 0 registros duplicados na base_originacao!


In [5]:
conn = sqlite3.connect('teste_ailos.db')

base_originacao = pd.read_sql_query("SELECT * FROM base_originacao", conn)
base_carteira = pd.read_sql_query("SELECT * FROM base_carteira", conn)

In [6]:
%reload_ext sql
%sql sqlite:///teste_ailos.db

# RESOLUÇÃO DO EXERCÍCIO EM SQL

### A) Calcule a quantidade de dias distintos DTCONTRATO e a soma dos VLCONTRATO dos contratos originados

In [7]:
%%sql
select * from base_originacao limit 5;

 * sqlite:///teste_ailos.db
Done.


index,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,VLPARCELA,QTPARCELAS
0,A,32,496314,677366,Financiamento,2021-10-13 00:00:00,4263.6,153.0,36.0
1,A,38,624412,644724,Empréstimo,2021-09-01 00:00:00,81.6,10.2,10.0
2,A,154,49834,349384,Empréstimo,2020-08-25 00:00:00,9761.4,204.0,72.0
3,A,186,398914,444074,Empréstimo,2021-01-08 00:00:00,4314.6,163.2,36.0
4,A,430,549582,372569,Empréstimo,2020-10-02 00:00:00,5548.8,183.6,48.0


In [8]:
%%sql
select * from base_carteira limit 5;

 * sqlite:///teste_ailos.db
Done.


index,DTBASE,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,SDDEVEDOR,QTDIASATRASO,SDATRASO,VLPROVISAO
0,2021-01-31 00:00:00,A,154,49834,349384,Empréstimo,2020-08-25 00:00:00,9761.4,9557.4,0.0,0.0,30.6
1,2021-01-31 00:00:00,A,186,398914,444074,Empréstimo,2021-01-08 00:00:00,4314.6,4365.6,0.0,0.0,81.6
2,2021-01-31 00:00:00,A,430,549582,372569,Empréstimo,2020-10-02 00:00:00,5548.8,5273.4,0.0,0.0,20.4
3,2021-01-31 00:00:00,A,512,217818,223696,Empréstimo,2020-01-31 00:00:00,3111.0,2539.8,0.0,0.0,10.2
4,2021-01-31 00:00:00,A,754,546819,435007,Empréstimo,2020-12-23 00:00:00,601.8,601.8,19.0,601.8,10.2


In [9]:
%%sql
select 
    count(distinct DTCONTRATO) as qnt_dias, round(sum(VLCONTRATO)) as sum_contrato
from
    base_originacao;

 * sqlite:///teste_ailos.db
Done.


qnt_dias,sum_contrato
921,32816287.0


### B) Para DTBASE = 31/12/2021 da base "base_carteira.csv", faça a soma dos SDDEVEDOR e a média simples de QTDIASATRASO

In [10]:
%%sql
select 
    sum(SDDEVEDOR) as sum_saldo_dev, avg(QTDIASATRASO) as avg_dias_atraso
from 
    base_carteira
where
    DTBASE >= '2021-12-31';

 * sqlite:///teste_ailos.db
Done.


sum_saldo_dev,avg_dias_atraso
18183172.800000023,31.502131287297527


# RESOLUÇÃO DO EXERCÍCIO EM PYTHON

### A) Calcule a quantidade de dias distintos DTCONTRATO e a soma dos VLCONTRATO dos contratos originados

In [11]:
base_originacao.head()

,index,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,VLPARCELA,QTPARCELAS
0,0,A,32,496314,677366,Financiamento,2021-10-13 00:00:00,4263.6,153.0,36.0
1,1,A,38,624412,644724,Empréstimo,2021-09-01 00:00:00,81.6,10.2,10.0
2,2,A,154,49834,349384,Empréstimo,2020-08-25 00:00:00,9761.4,204.0,72.0
3,3,A,186,398914,444074,Empréstimo,2021-01-08 00:00:00,4314.6,163.2,36.0
4,4,A,430,549582,372569,Empréstimo,2020-10-02 00:00:00,5548.8,183.6,48.0


In [12]:
Qnt_dias = base_originacao.DTCONTRATO.unique().shape[0]
Sum_contrato = round(base_originacao.VLCONTRATO.sum(), 2)

print('A quantidade de dias distintos é: ', Qnt_dias)
print('A soma dos contratos é: ', Sum_contrato)

A quantidade de dias distintos é:  921
A soma dos contratos é:  32816286.6


### B) Para DTBASE = 31/12/2021 da base "base_carteira.csv", faça a soma dos SDDEVEDOR e a média simples de QTDIASATRASO

In [13]:
base_carteira.head()

sum_saldo_dev = base_carteira[base_carteira.DTBASE >= '2021-12-31'].SDDEVEDOR.sum()
avg_dias_atraso = base_carteira[base_carteira.DTBASE >= '2021-12-31'].QTDIASATRASO.mean()

print('A soma do saldo devedor é: ', sum_saldo_dev)
print('A média dos dias de atraso é: ', avg_dias_atraso)

A soma do saldo devedor é:  18183172.799999997
A média dos dias de atraso é:  31.502131287297527
